# Day 3 - Conversational AI - aka Chatbot!

In [28]:
# Import required libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display

In [29]:
import gradio as gr

In [30]:
# Load environment variables from .env file
load_dotenv(override=True)

# API keys from environment
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_model = os.getenv('GOOGLE_MODEL')
ollama_base_url = os.getenv('OLLAMA_BASE_URL')
ollama_api_key = os.getenv('OLLAMA_API_KEY')
ollama_model = os.getenv('OLLAMA_MODEL', 'deepseek-v3.1:671b-cloud')

# Verify API keys
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[:8]}...")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key loaded: {anthropic_api_key[:7]}...")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key loaded: {google_api_key[:2]}...")
else:
    print("Google API Key not set")

if ollama_base_url:
    print(f"Ollama configured at: {ollama_base_url}")

OpenAI API Key loaded: sk-proj-...
Anthropic API Key loaded: sk-ant-...
Google API Key loaded: AI...
Ollama configured at: http://192.168.80.200:11434


In [31]:
# Initialize API clients
openai_client = OpenAI(api_key=openai_api_key)
claude_client = anthropic.Anthropic(api_key=anthropic_api_key)
google.generativeai.configure(api_key=google_api_key)

# Initialize Ollama client (uses OpenAI-compatible API)
ollama_client = OpenAI(
    base_url=f"{ollama_base_url}/v1",
    api_key=ollama_api_key
)

print("All clients initialized successfully")

All clients initialized successfully


In [32]:
def call_ollama(prompt, stream=False):
    """Call Ollama model with OpenAI-compatible API"""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    
    response = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        stream=stream
    )
    
    if stream:
        for chunk in response:
            if chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end='', flush=True)
    else:
        return response.choices[0].message.content

In [19]:
# Again, I'll be in scientist-mode and change this global during the lab

system_message = "You are a helpful assistant"

## And now, writing a new callback

We now need to write a function called:

`chat(message, history)`

Which will be a callback function we will give gradio.

### The job of this function

Take a message, take the prior conversation, and return the response.


In [25]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    stream = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        stream = True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [13]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

Closing server running on port: 7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3db28181c41d12cdae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


El historial es: 
[]
Los mensajes son: 
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hola'}]
El historial es: 
[{'role': 'user', 'content': 'Hola'}, {'role': 'assistant', 'content': '¡Hola! 😊 ¿En qué puedo ayudarte hoy?'}]
Los mensajes son: 
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hola'}, {'role': 'assistant', 'content': '¡Hola! 😊 ¿En qué puedo ayudarte hoy?'}, {'role': 'user', 'content': 'quiero comprar una corbata'}]


## OK let's keep going!

Using a system message to add context, and to give an example answer.. this is "one shot prompting" again

In [20]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [21]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://163b4e221a166c122f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


El historial es: 
[]
Los mensajes son: 
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get."}, {'role': 'user', 'content': 'Hola'}]
El historial es: 
[{'role': 'user', 'content': 'Hola'}, {'role': 'assistant', 'content': '¡Hola! Bienvenido/a a nuestra tienda. ¿En qué puedo ayudarle hoy? Por ejemplo, tenemos muchas ofertas especiales en gorras y otras prendas. ¿Hay algo que le interese ver?'}]
Los mensajes son: 
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, 

In [22]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [24]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://357de3778122269866.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


El historial es: 
[]
Los mensajes son: 
[{'role': 'system', 'content': "You are a helpful assistant in a clothes store. You should try to gently encourage the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. For example, if the customer says 'I'm looking to buy a hat', you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'Encourage the customer to buy hats if they are unsure what to get.\nIf the customer asks for shoes, you should respond that shoes are not on sale today, but remind the customer to look at hats!"}, {'role': 'user', 'content': 'hola'}]
El historial es: 
[{'role': 'user', 'content': 'hola'}, {'role': 'assistant', 'content': '¡Hola! ¡Bienvenido/a a nuestra tienda! ¿En qué puedo ayudarte hoy? ¿Buscas algo en especial? Tenemos una fantástica promoción en sombreros, con un 60% de descuento.'}]
Los mensajes son: 
[{'role': 'system', 'content': "You are a helpful assist

In [33]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = ollama_client.chat.completions.create(
        model=ollama_model,
        messages=messages,
        stream = True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [34]:
gr.close_all()
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://bcf3359b474f8b6df1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
